In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
from math import pi, cos, sin, sqrt, atan2
import networkx as nx 
import random
import os
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
file_no = 13
df_collapsed    = pl.read_parquet(f'../../data/stanford/facebook/348.edges_collapsed_edges.{file_no}.parquet')
df_edge_arc_pos = pl.read_parquet(f'../../data/stanford/facebook/348.edges_arc_pos.{file_no}.parquet')
df_collapsed.sample(10)

In [ ]:
df_edge_arc_pos.sample(1)

In [4]:
def extractCircles(_df_):
    circles, circle_lu = [], {}
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        circle_lu[k] = len(circles)
        circles.append(k)
    return circles, circle_lu
def validatePositions(_df_, clearance=10): # edge_arc_pos
    # Extract the circles first
    circles, circle_lu = extractCircles(_df_)
    # Then make sure the positions don't fall within a circle & that they have some clearance
    for k, k_df in _df_.group_by(['x','y', 'x_circle', 'y_circle', 'r_circle']):
        _xy_     = (k[0], k[1])
        _uv_     = rt.unitVector((k[2:4], _xy_))
        _xy_end_ = (_xy_[0]+ clearance*_uv_[0], _xy_[1]+ clearance*_uv_[1])
        _my_circle_ = k[2:5]
        for c in circles:
            if c == _my_circle_: continue
            _length_ = rt.segmentLength((_xy_,c))
            if _length_ < c[2] + clearance:
                raise Exception(f'node key "{k_df["node_key"]}" with radius circle {c}')
            _length_ = rt.segmentLength((_xy_end_, c))

def extents(_df_): # edge_arc_pos -- assumes that the coordinates are already in screen space
    xmax, ymax = 10.0, 10.0
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        xmax, ymax = max(xmax, k[0]+k[2]), max(ymax, k[1]+k[2])
    for k, k_df in _df_.group_by(['x','y']):
        xmax, ymax = max(xmax, k[0]), max(ymax, k[1])
    return (0.0, 0.0, xmax, ymax)

validatePositions(df_edge_arc_pos)
_ext_ = extents(df_edge_arc_pos)

In [ ]:
x_ins, y_ins = 50, 50
# Base Circles
svg_base = [f'<svg x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}">']
svg_base.append(f'<rect x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}" fill="#ffffff" />')
circles, circles_lu = extractCircles(df_edge_arc_pos)
for i in range(len(circles)):
    cx, cy, r = circles[i]
    svg_base.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3" />')
# Entry / Exit Points
svg_pts = []
for k, k_df in df_edge_arc_pos.group_by(['node_key', 'x', 'y', 'x_circle', 'y_circle', 'r_circle']):
    if '_to_' in k[0]: _color_, _fill_ = '#ff0000', 'none'
    else:              _color_, _fill_ = '#00ff00', '#00b000'
    _xy_, _cxy_ = k[1:3], k[3:5]
    svg_pts.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="4" fill="{_fill_}" stroke="{_color_}" stroke-width="0.5"/>')
    _uv_        = rt.unitVector((_cxy_, _xy_))
    svg_pts.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy_[0]+10*_uv_[0]}" y2="{_xy_[1]+10*_uv_[1]}" stroke="#404040" stroke-width="1.0"/>')
# Delauney Triangulations
_box_ = [(_ext_[0],_ext_[1]),(_ext_[0],_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[1])]
voronoi_polys       = rt.isedgarVoronoi(circles, Box=_box_, use_circle_radius=True)
voronoi_point_polys = rt.isedgarVoronoi(circles, Box=_box_, use_circle_radius=False)
svg_voronoi = []
for i in range(len(voronoi_polys)):
    d = f'M {voronoi_polys[i][0][0]} {voronoi_polys[i][0][1]} '
    for j in range(1, len(voronoi_polys[i])): d += f'L {voronoi_polys[i][j][0]} {voronoi_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1"/>')
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="#000000" stroke-width="0.1"/>')
_seen_ = set()
svg_voronoi_circle_markers = []
for i in range(len(voronoi_polys)):
    for j in range(len(voronoi_polys[i])):
        x, y = voronoi_polys[i][j]
        if (x,y) not in _seen_: svg_voronoi_circle_markers.append(f'<circle cx="{x}" cy="{y}" r="{4.0+random.random()*4.0}" stroke-width="0.1" stroke="#000000" fill="none"/>')
        _seen_.add((x,y))
svg_voronoi_point = []
for i in range(len(voronoi_point_polys)):
    d = f'M {voronoi_point_polys[i][0][0]} {voronoi_point_polys[i][0][1]} '
    for j in range(1, len(voronoi_point_polys[i])): d += f'L {voronoi_point_polys[i][j][0]} {voronoi_point_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi_point.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1"/>')
_seen_ = set()
for i in range(len(voronoi_point_polys)):
    _color_ = rt.co_mgr.getColor(i)
    for j in range(len(voronoi_point_polys[i])):
        x, y = voronoi_point_polys[i][j]
        # if (x,y) not in _seen_: svg_voronoi_point.append(f'<circle cx="{x}" cy="{y}" r="{5.0 + random.random()*10.0}" stroke="{_color_}" stroke-width="2.0" fill="none"/>')
        _seen_.add((x,y))
        
# Print the number of unique points
vpoints, vpoints_pt = set(), set()
for i in range(len(voronoi_polys)):
    for j in range(len(voronoi_polys[i])):        vpoints   .add(voronoi_polys[i][j])
    for j in range(len(voronoi_point_polys[i])):  vpoints_pt.add(voronoi_point_polys[i][j])
print(len(vpoints), len(vpoints_pt))

#rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi_point)+''.join(svg_voronoi)+'</svg>'])
rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+''.join(svg_voronoi_circle_markers)+'</svg>'])

In [6]:
# For every point, find the three closest circles
# ... this doesn't work...
three_to_xys = {}
for _xy_ in vpoints:
    def dToCircle(_coord_, circle_i): return sqrt((_coord_[0]-circles[circle_i][0])**2 + (_coord_[1]-circles[circle_i][1])**2) - circles[circle_i][2]
    _dist_    = [dToCircle(_xy_, 0), dToCircle(_xy_, 1), dToCircle(_xy_, 2)]
    _closest_ = [0,1,2]
    for i in range(3, len(circles)):
        d = dToCircle(_xy_, i)
        for j in range(3):
            if d < _dist_[j]:
                _closest_[j] = i
                _dist_   [j] = d
                break
    _closest_.sort()
    _as_tuple_ = tuple(_closest_)
    if _as_tuple_ not in three_to_xys: three_to_xys[_as_tuple_] = set()
    three_to_xys[_as_tuple_].add(_xy_)
threes_svg = []
for _tuple_ in three_to_xys:
    _set_ = three_to_xys[_tuple_]
    if len(_set_) >= 3:
        _x_, _y_ = 0.0, 0.0
        for _xy_ in _set_: _x_, _y_ = _x_ + _xy_[0], _y_ + _xy_[1]
        _x_, _y_ = _x_ / len(_set_), _y_ / len(_set_)
        threes_svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="4" fill="none" stroke="#000000" stroke-dasharray="1 0.2" />')
# rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+''.join(threes_svg)+'</svg>'])

In [7]:
xys_to_polys = {}
for _xy_ in vpoints:
    xys_to_polys[_xy_] = set()
    for i in range(len(voronoi_polys)):
        for j in range(len(voronoi_polys[i])):
            if _xy_ == voronoi_polys[i][j]: xys_to_polys[_xy_].add(i)
svg_points_with_less_than_3 = []
for _xy_ in xys_to_polys:
    if len(xys_to_polys[_xy_]) < 3: svg_points_with_less_than_3.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="4" fill="none" stroke="#000000" stroke-dasharray="1 0.2" />')
#rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+''.join(svg_points_with_less_than_3)+'</svg>'])

In [ ]:
point_to_polys          = {}
for i in range(len(voronoi_polys)):
    _poly_ = voronoi_polys[i]
    for _xy_ in _poly_:
        if _xy_ not in point_to_polys: point_to_polys[_xy_] = set()
        point_to_polys[_xy_].add(i)
poly_connects           = {'__fm__':[], '__to__':[]}
overlapping_segments_lu = {}
for i in range(len(voronoi_polys)):
    _poly0_ = voronoi_polys[i]
    for j in range(len(voronoi_polys)):
        if i >= j: continue
        _poly1_     = voronoi_polys[j]
        shared_edge = False
        for k in range(len(_poly0_)):
            _seg0_ = (_poly0_[k], _poly0_[(k+1)%len(_poly0_)])
            for l in range(len(_poly1_)):
                _seg1_ = (_poly1_[l], _poly1_[(l+1)%len(_poly1_)])
                if rt.segmentsOverlap(_seg0_, _seg1_): 
                    shared_edge = True
                    overlapping_segments_lu[(i,j)] = (_seg0_, _seg1_)
                    break
            if shared_edge: break
        if shared_edge:
            poly_connects['__fm__'].append(i)
            poly_connects['__to__'].append(j)

svg_cycles = []
for _xy_ in point_to_polys:
    if len(point_to_polys[_xy_]) >= 3:
        svg_cycles.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="5" fill="none" stroke="#00af00"/>')

g_poly = rt.createNetworkXGraph(pl.DataFrame(poly_connects), [('__fm__','__to__')])
for x in nx.simple_cycles(g_poly, 3):
    _as_list_ = sorted(list(x))
    poly_i, poly_j, poly_k = _as_list_[0], _as_list_[1], _as_list_[2]
    _inter_ = set(voronoi_polys[poly_i]) & set(voronoi_polys[poly_j]) & set(voronoi_polys[poly_k])
    if len(_inter_) != 0:
        _xy_ = _inter_.pop()
        svg_cycles.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="5" fill="none" stroke="#00af00"/>')
    else:
        _segs0_ = overlapping_segments_lu[(poly_i,poly_j)]
        _segs1_ = overlapping_segments_lu[(poly_i,poly_k)]
        _segs2_ = overlapping_segments_lu[(poly_j,poly_k)]
        _xy0_   = rt.intersectionPoint(_segs0_[0],_segs1_[0])
        _xy1_   = rt.intersectionPoint(_segs0_[0],_segs2_[0])
        _xy2_   = rt.intersectionPoint(_segs1_[0],_segs2_[0])
        for i in range(3):
            if   i == 0: _xy_ = _xy0_
            elif i == 1: _xy_ = _xy1_
            else:        _xy_ = _xy2_
            _l_ = 2
            svg_cycles.append(f'<line x1="{_xy_[0]-_l_}" y1="{_xy_[1]-_l_}" x2="{_xy_[0]+_l_}" y2="{_xy_[1]+_l_}" stroke="#ff0000"/>')
            svg_cycles.append(f'<line x1="{_xy_[0]+_l_}" y1="{_xy_[1]-_l_}" x2="{_xy_[0]-_l_}" y2="{_xy_[1]+_l_}" stroke="#ff0000"/>')
        _cx_, _cy_, _r_ = rt.threePointCircle(_xy0_, _xy1_, _xy2_)
        svg_cycles.append(f'<circle cx="{_cx_}" cy="{_cy_}" r="{_r_}" fill="none" stroke="#ff0000" />')

rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+''.join(svg_cycles)+'</svg>'])